<a href="https://colab.research.google.com/github/typleasant22/big-data-challenge/blob/main/Video_Games.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:13 https://developer.

In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-03-05 17:01:37--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.61MB/s    in 0.5s    

2022-03-05 17:01:38 (1.61 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
from pyspark import SparkFiles

In [38]:
from google.colab import data_table

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType


In [11]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [26]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
videogames_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep = "\t", header = True, inferSchema=True)

# Show DataFrame
videogames_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [28]:
video_reviews_df= videogames_df.select(["customer_id","product_id","review_date","review_body"])
video_reviews_df.show()

+-----------+----------+-----------+--------------------+
|customer_id|product_id|review_date|         review_body|
+-----------+----------+-----------+--------------------+
|   12039526|B001CXYMFS| 2015-08-31|Used this for Eli...|
|    9636577|B00M920ND6| 2015-08-31|Loved it,  I didn...|
|    2331478|B0029CSOD2| 2015-08-31|poor quality work...|
|   52495923|B00GOOSV98| 2015-08-31|nice, but tend to...|
|   14533949|B00Y074JOM| 2015-08-31|Great amiibo, gre...|
|    2377552|B002UBI6W6| 2015-08-31|The remote consta...|
|   17521011|B008XHCLFO| 2015-08-31|I have a 2012-201...|
|   19676307|B00BRA9R6A| 2015-08-31|Perfect, kids lov...|
|     224068|B009EPWJLA| 2015-08-31|            Excelent|
|   48467989|B0000AV7GB| 2015-08-31|Slippery but expe...|
|     106569|B00008KTNN| 2015-08-31|Love the game. Se...|
|   48269642|B000A3IA0Y| 2015-08-31|Does not fit prop...|
|   52738710|B0053BQN34| 2015-08-31|This was way too ...|
|   10556786|B002I0HIMI| 2015-08-31|Works great good ...|
|    2963837|B

In [30]:
video_reviews_df= videogames_df.groupBy("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
video_reviews_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   48670265|             1|
|   49103216|             2|
|    1131200|             1|
|   43076447|             2|
|   46261368|             1|
|    4883305|             5|
|   41192649|             1|
|   40985731|             7|
|   10437900|             2|
|   22245671|             1|
|    2574873|             1|
|    4696154|             1|
|    5621202|             1|
|    5871933|             2|
|   44089812|             1|
|    2845910|             1|
|    5274369|             1|
|   39069693|             2|
|     137793|             1|
|   31914942|             3|
+-----------+--------------+
only showing top 20 rows



In [34]:
video_reviews_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [ ]:
##RDS
#Config settings for RDS

## IMPORTANT : PLEASE ADD YOUR OWN CREDENTIALS FOR RDS ##

mode = "append"
jdbc_url="jdbc:postgresql://homework.cqekbhx09q1f.us-east-1.rds.amazonaws.com"
config = {"user":"username", 
          "password": "Password", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to review_id table in RDS

video_reviews_df.write.jdbc(url=jdbc_url, table="customer_id", mode=mode, properties= config)

In [ ]:
video_reviews_df.write.jdbc(url=jdbc_url, table="review_id", mode=mode, properties= config)

In [ ]:
video_reviews_df.write.jdbc(url=jdbc_url, table="music_review_table", mode=mode, properties= config)